In [4]:
pip install -r requirements.txt

  Using cached scipy-1.11.4.tar.gz (56.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + meson setup C:\Users\jeffr\AppData\Local\Temp\pip-install-ey4yqihg\scipy_b53b2d22c1f947cfbfcff6253fe83934 C:\Users\jeffr\AppData\Local\Temp\pip-install-ey4yqihg\scipy_b53b2d22c1f947cfbfcff6253fe83934\.mesonpy-sdjp71tg -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\jeffr\AppData\Local\Temp\pip-install-ey4yqihg\scipy_b53b2d22c1f947cfbfcff6253fe83934\.mesonpy-sdjp71tg\meson-python-native-file.ini
      The Meson build system
      Version: 1.9.1
      Source dir: C:\Users\jeffr\AppData\Local\Temp\pip-install-ey4yqihg\scipy_b53b2d22c1f947cfbfcff6253fe83934
      Build dir: C:\Users\jeffr\AppData\Local\Temp\pip-install-ey4yqihg\scipy_b53b2d22c1f947cfbfcff6253fe83934\.mesonpy-sdjp71tg
      Build type: native build
      Project name: SciPy
      Project version: 1.11.4
      
      ..\meson.build:1:0: ER

## Read the raw Excel files

The Excel files are stored in the `datasets` folder inside this repository:
- `sales.xlsx`
- `UserReviews.xlsx`
- `ExpertReviews.xlsx`
- `metaClean43Brightspace.xlsx`

First we check the current working directory and the contents of the `datasets` folder, then we load each file into a pandas DataFrame.


In [14]:
import numpy as np
import pandas as pd
import plotnine as plotnine
from plotnine import * # generally not a good thing to do to import everything from a package. However it's ok for visualization purposes in an analysis.
import os
import scipy
import warnings
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

warnings.filterwarnings("ignore")  # to keep the output clean

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [ ]:
# we can get the current working directory with the following command
os.getcwd()

# we can then list the files and directories in the dataset directory
os.listdir(os.path.join('datasets')) # independent of the platform (unix, windows)



'c:\\Users\\jeffr\\OneDrive\\Documenten\\Digital Drive Business\\AI methods for business\\project map\\bjj'

['ExpertReviews.xlsx',
 'metaClean43Brightspace.xlsx',
 'sales.xlsx',
 'UserReviews.xlsx']

['ExpertReviews.xlsx', 'metaClean43Brightspace.xlsx', 'sales.xlsx', 'UserReviews.xlsx']


In [17]:
# define file paths relative to the notebook
data_folder = "datasets"

sales_path = os.path.join(data_folder, "sales.xlsx")
userreviews_path = os.path.join(data_folder, "UserReviews.xlsx")
expertreviews_path = os.path.join(data_folder, "ExpertReviews.xlsx")
meta_path = os.path.join(data_folder, "metaClean43Brightspace.xlsx")

# read the Excel files
sales_df = pd.read_excel(sales_path)
UserReviews = pd.read_excel(userreviews_path)
ExpertReviews = pd.read_excel(expertreviews_path)
meta_file_clean = pd.read_excel(meta_path)

# quick overview of shapes
for name, df in [
    ("sales_df", sales_df),
    ("UserReviews", UserReviews),
    ("ExpertReviews", ExpertReviews),
    ("meta_file_clean", meta_file_clean),
]:
    print(f"{name}: {df.shape}")


sales_df: (30612, 16)
UserReviews: (319662, 7)
ExpertReviews: (238973, 5)
meta_file_clean: (11364, 13)


## Quick overview of the tables

Here we look at:
- the first rows of each table
- the data types of the columns
- how many missing values there are

We define a small helper function to print this information.


In [18]:
def quick_overview(df, name, n_head=5):
    """
    Print a quick overview of a DataFrame.
    """
    print("=" * 80)
    print(f"Overview of {name}")
    print("- Shape:", df.shape)
    print("- First rows:")
    display(df.head(n_head))
    print("- Dtypes:")
    print(df.dtypes)
    print("- Fraction of missing values (top 15):")
    print(df.isna().mean().sort_values(ascending=False).head(15))
    print()

quick_overview(sales_df, "sales_df")
quick_overview(UserReviews, "UserReviews")
quick_overview(ExpertReviews, "ExpertReviews")
quick_overview(meta_file_clean, "meta_file_clean")


Overview of sales_df
- Shape: (30612, 16)
- First rows:


,year,release_date,title,genre,international_box_office,domestic_box_office,worldwide_box_office,production_budget,Unnamed: 8,opening_weekend,theatre_count,avg run per theatre,runtime,keywords,creative_type,url
0,2000,January 1st,Bakha Satang,Drama,76576.0,NaN,76576.0,NaN,NaN,NaN,NaN,NaN,129.0,NaN,Contemporary Fiction,https://www.the-numbers.com/movie/Bakha-Satang...
1,2001,January 12th,Antitrust,Thriller/Suspense,6900000.0,10965209.0,17865209.0,30000000.0,NaN,5486209.0,2433.0,3.1,NaN,NaN,Contemporary Fiction,https://www.the-numbers.com/movie/Antitrust
2,2000,January 28th,Santitos,NaN,NaN,378562.0,NaN,NaN,NaN,NaN,NaN,NaN,105.0,NaN,NaN,https://www.the-numbers.com/movie/Santitos
3,2002,2002 (Wide) by,Frank McKlusky C.I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.the-numbers.com/movie/Frank-McKlus...
4,2002,January 25th,A Walk to Remember,Drama,4833792.0,41227069.0,46060861.0,11000000.0,NaN,12177488.0,2411.0,5.3,NaN,Coming of Age,Contemporary Fiction,https://www.the-numbers.com/movie/Walk-to-Reme...


- Dtypes:
year                          int64
release_date                 object
title                        object
genre                        object
international_box_office    float64
domestic_box_office         float64
worldwide_box_office        float64
production_budget           float64
Unnamed: 8                  float64
opening_weekend             float64
theatre_count               float64
avg run per theatre         float64
runtime                     float64
keywords                     object
creative_type                object
url                          object
dtype: object
- Fraction of missing values (top 15):
Unnamed: 8                  1.000000
production_budget           0.853652
opening_weekend             0.642983
avg run per theatre         0.642232
theatre_count               0.641872
domestic_box_office         0.611786
keywords                    0.591108
worldwide_box_office        0.295211
international_box_office    0.295211
runtime                     

,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot
0,https://www.metacritic.com/movie/bronson,8,'Longbottom94',"'Apr 25, 2013'",'Many have dismissed this film for not explor...,2,2
1,https://www.metacritic.com/movie/bronson,9,'MartinB',"'Oct 13, 2009'",'Anyone who doesn t like this movie simply ju...,0,1
2,https://www.metacritic.com/movie/bronson,10,'Jaakko',"'Jul 19, 2012'",'Not sure what to think at this film at first...,1,1
3,https://www.metacritic.com/movie/bronson,6,'CapoR',"'Oct 13, 2009'",'Nicely portrayed but it lacks the elements t...,0,1
4,https://www.metacritic.com/movie/bronson,8,'OrwellB.',"'Oct 10, 2009'",'Bronson is more than entertainment. It is ar...,0,0


- Dtypes:
url          object
idvscore     object
reviewer     object
dateP        object
Rev          object
thumbsUp     object
thumbsTot    object
dtype: object
- Fraction of missing values (top 15):
thumbsUp     0.011199
thumbsTot    0.011187
Rev          0.010677
dateP        0.010677
reviewer     0.010658
idvscore     0.010649
url          0.000000
dtype: float64

Overview of ExpertReviews
- Shape: (238973, 5)
- First rows:


,url,idvscore,reviewer,dateP,Rev
0,https://www.metacritic.com/movie/bronson,100.0,"""Andrew O'Hehir""",None,'Bronson owes a little or a lot to Kubrick s ...
1,https://www.metacritic.com/movie/bronson,90.0,'A.O. Scott',None,'Bronson invites you to admire its protagonis...
2,https://www.metacritic.com/movie/bronson,90.0,None,None,'Whether it s Peterson/Bronson s more theatri...
3,https://www.metacritic.com/movie/bronson,83.0,'Noel Murray',None,'There are two Bronsons on display here: the ...
4,https://www.metacritic.com/movie/bronson,80.0,'Joshua Rothkopf',None,'Refn has somehow found his way to an authent...


- Dtypes:
url          object
idvscore    float64
reviewer     object
dateP        object
Rev          object
dtype: object
- Fraction of missing values (top 15):
idvscore    0.000008
dateP       0.000008
reviewer    0.000008
Rev         0.000008
url         0.000000
dtype: float64

Overview of meta_file_clean
- Shape: (11364, 13)
- First rows:


,url,title,studio,rating,runtime,cast,director,genre,summary,awards,metascore,userscore,RelDate
0,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,Hotwire Productions,| Not Rated,83.0,NaN,Lynn Hershman-Leeson,Documentary,NaN,NaN,70,NaN,2011-06-01
1,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,Paramount Pictures,| PG-13,104.0,"John Gallagher Jr.,John Goodman,Mary Elizabeth...",Dan Trachtenberg,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror","Waking up from a car accident, a young woman (...","#18MostDiscussedMovieof2016 , #1MostSharedMovi...",76,7.7,2016-03-11
2,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,Click Star,| R,82.0,"Jonah Hill,Morgan Freeman,Paz Vega",Brad Silberling,"Drama,Comedy,Romance",While researching a role as a supermarket mana...,NaN,54,5.8,2006-12-01
3,https://www.metacritic.com/movie/10-years,10 Years,Anchor Bay Entertainment,| R,100.0,"Channing Tatum,Chris Pratt,Jenna Dewan",Jamie Linden,"Drama,Comedy,Romance",NaN,NaN,61,6.9,2012-09-14
4,https://www.metacritic.com/movie/100-bloody-acres,100 Bloody Acres,Music Box Films,| Not Rated,91.0,NaN,Cameron Cairnes,"Horror,Comedy",Reg and Lindsay run an organic fertilizer busi...,NaN,63,7.5,2013-06-28


- Dtypes:
url                  object
title                object
studio               object
rating               object
runtime             float64
cast                 object
director             object
genre                object
summary              object
awards               object
metascore             int64
userscore           float64
RelDate      datetime64[ns]
dtype: object
- Fraction of missing values (top 15):
awards       0.613956
summary      0.518919
cast         0.325766
userscore    0.185234
rating       0.093893
studio       0.030799
runtime      0.022439
genre        0.001760
director     0.001232
title        0.000000
url          0.000000
metascore    0.000000
RelDate      0.000000
dtype: float64



## Cleaning functions (reused from the Database Management project)

We now define helper functions to clean the raw data:

1. `safe_column_access`  
   Safely access a column. If the column does not exist, we return a Series filled with a default value.

2. `get_numeric_value` and `clean_numeric_columns`  
   Convert messy numeric columns (for example with dollar signs or commas) to proper numeric dtype.

3. `calculate_total_box_office`  
   Create a new column `total_box_office_sales` from the different box office columns.

4. `clean_reviewer_name` and `clean_reviewer_column`  
   Make reviewer names consistent and replace empty names with `"Unknown Reviewer"`.

5. `clean_review_dates`  
   Parse review dates to proper datetime format.

6. `clean_sales_dates`  
   Create a cleaned release date column for the sales table.


In [19]:
# 1. Safe access helper ---------------------------------------------

def safe_column_access(df, column_name, default=np.nan):
    """
    Return df[column_name] if the column exists,
    otherwise return a Series filled with a default value.
    """
    if column_name in df.columns:
        return df[column_name]
    else:
        return pd.Series(default, index=df.index)


# 2. Numeric cleaning helpers ---------------------------------------

def get_numeric_value(column):
    """
    Clean up numbers that might have dollar signs or other text.

    Example input: "$1,234,567"
    Example output: 1234567.0
    """
    # convert to string, remove everything that is not a digit or dot,
    # then convert to numeric
    cleaned = (
        column.astype(str)
        .str.replace(r"[^0-9.]", "", regex=True)
    )
    return pd.to_numeric(cleaned, errors="coerce")


def clean_numeric_columns(df, columns):
    """
    Convert multiple columns to numeric dtype, coercing errors to NaN.
    """
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors="coerce")


# 3. Total box office from the sales table --------------------------

def calculate_total_box_office(sales_df):
    """
    Calculate total box office sales from the available box office columns.

    Logic:
    - If 'worldwide_box_office' is available, use that.
    - Otherwise, add 'domestic_box_office' and 'international_box_office'.
    - If both domestic and international are missing, the total is NaN.
    """
    print("Calculating total_box_office_sales in sales_df...")

    worldwide = get_numeric_value(
        safe_column_access(sales_df, "worldwide_box_office", np.nan)
    )
    domestic = get_numeric_value(
        safe_column_access(sales_df, "domestic_box_office", np.nan)
    )
    international = get_numeric_value(
        safe_column_access(sales_df, "international_box_office", np.nan)
    )

    combined = domestic.fillna(0) + international.fillna(0)
    both_missing = domestic.isna() & international.isna()
    combined = combined.mask(both_missing, np.nan)

    sales_df["total_box_office_sales"] = worldwide.where(
        worldwide.notna(), combined
    )


In [20]:
# 4. Reviewer name cleaning -----------------------------------------

def clean_reviewer_name(name):
    """
    Standardize reviewer names.

    - If the name is missing or one of several 'empty' strings,
      return 'Unknown Reviewer'.
    - Otherwise return a stripped string version.
    """
    if pd.isna(name) or name is None:
        return "Unknown Reviewer"

    str_name = str(name).strip()
    if str_name in ["", "nan", "NaN", "None", "null", "NULL"]:
        return "Unknown Reviewer"

    return str_name


def clean_reviewer_column(df):
    """
    Apply clean_reviewer_name to the 'reviewer' column if present.
    """
    if "reviewer" in df.columns:
        df["reviewer"] = df["reviewer"].apply(clean_reviewer_name)


# 5. Review date cleaning -------------------------------------------

def clean_review_dates(df):
    """
    Convert the 'dateP' column (if present) to datetime.
    """
    if "dateP" in df.columns:
        df["dateP"] = pd.to_datetime(df["dateP"], errors="coerce")


# 6. Sales date cleaning --------------------------------------------

def clean_sales_dates(sales_df):
    """
    Create a cleaned release date column for the sales dataframe.

    - Try to parse the 'release_date' column directly.
    - If parsing fails but a 'year' is available, use July 1st of that year
      as a fallback value.
    """
    if "release_date" not in sales_df.columns:
        return

    # try to parse the raw release_date column
    clean_dates = pd.to_datetime(sales_df["release_date"], errors="coerce")

    # fallback using the 'year' column if we have it
    if "year" in sales_df.columns:
        year_numeric = pd.to_numeric(sales_df["year"], errors="coerce")
        missing_mask = clean_dates.isna() & year_numeric.notna()

        # build a fake date "YYYY-07-01" as a neutral mid-year date
        fallback_dates = pd.to_datetime(
            year_numeric[missing_mask].astype(int).astype(str) + "-07-01",
            errors="coerce"
        )
        clean_dates.loc[missing_mask] = fallback_dates

    sales_df["release_date_clean"] = clean_dates


## Apply cleaning to the raw tables

Now we apply the cleaning functions:

- `sales_df`
  - clean box office columns and create `total_box_office_sales`
  - clean important numeric columns
  - create a cleaned release date

- `UserReviews` and `ExpertReviews`
  - clean reviewer names
  - convert `dateP` to datetime
  - clean a selection of numeric LIWC columns

- `meta_file_clean`
  - parse the release date column `RelDate` to datetime


In [21]:
# 1. Clean the sales table ------------------------------------------

# columns in sales_df that should be numeric (you can add more later)
sales_numeric_cols = [
    "production_budget",
    "opening_weekend",
    "theatre_count",
    "avg run per theatre",
    "runtime",
]

clean_numeric_columns(sales_df, sales_numeric_cols)
calculate_total_box_office(sales_df)
clean_sales_dates(sales_df)

quick_overview(sales_df, "sales_df after cleaning")


Calculating total_box_office_sales in sales_df...
Overview of sales_df after cleaning
- Shape: (30612, 18)
- First rows:


,year,release_date,title,genre,international_box_office,domestic_box_office,worldwide_box_office,production_budget,Unnamed: 8,opening_weekend,theatre_count,avg run per theatre,runtime,keywords,creative_type,url,total_box_office_sales,release_date_clean
0,2000,January 1st,Bakha Satang,Drama,76576.0,NaN,76576.0,NaN,NaN,NaN,NaN,NaN,129.0,NaN,Contemporary Fiction,https://www.the-numbers.com/movie/Bakha-Satang...,76576.0,2000-07-01
1,2001,January 12th,Antitrust,Thriller/Suspense,6900000.0,10965209.0,17865209.0,30000000.0,NaN,5486209.0,2433.0,3.1,NaN,NaN,Contemporary Fiction,https://www.the-numbers.com/movie/Antitrust,17865209.0,2001-07-01
2,2000,January 28th,Santitos,NaN,NaN,378562.0,NaN,NaN,NaN,NaN,NaN,NaN,105.0,NaN,NaN,https://www.the-numbers.com/movie/Santitos,378562.0,2000-07-01
3,2002,2002 (Wide) by,Frank McKlusky C.I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.the-numbers.com/movie/Frank-McKlus...,NaN,2002-07-01
4,2002,January 25th,A Walk to Remember,Drama,4833792.0,41227069.0,46060861.0,11000000.0,NaN,12177488.0,2411.0,5.3,NaN,Coming of Age,Contemporary Fiction,https://www.the-numbers.com/movie/Walk-to-Reme...,46060861.0,2002-07-01


- Dtypes:
year                                 int64
release_date                        object
title                               object
genre                               object
international_box_office           float64
domestic_box_office                float64
worldwide_box_office               float64
production_budget                  float64
Unnamed: 8                         float64
opening_weekend                    float64
theatre_count                      float64
avg run per theatre                float64
runtime                            float64
keywords                            object
creative_type                       object
url                                 object
total_box_office_sales             float64
release_date_clean          datetime64[ns]
dtype: object
- Fraction of missing values (top 15):
Unnamed: 8                  1.000000
production_budget           0.853652
opening_weekend             0.642983
avg run per theatre         0.642232
theatre_count  

In [22]:
# 2. Clean the review tables ----------------------------------------

# some typical numeric columns in ExpertReviews that must be numeric
expert_numeric_columns = [
    "WC",
    "Analytic",
    "Clout",
    "Authentic",
    "Tone",
    "WPS",
    "posemo",
    "negemo",
]

clean_numeric_columns(ExpertReviews, expert_numeric_columns)

# reviewer names and dates
clean_reviewer_column(UserReviews)
clean_reviewer_column(ExpertReviews)
clean_review_dates(UserReviews)
clean_review_dates(ExpertReviews)

quick_overview(UserReviews, "UserReviews after cleaning")
quick_overview(ExpertReviews, "ExpertReviews after cleaning")


Overview of UserReviews after cleaning
- Shape: (319662, 7)
- First rows:


,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot
0,https://www.metacritic.com/movie/bronson,8,'Longbottom94',2013-04-25,'Many have dismissed this film for not explor...,2,2
1,https://www.metacritic.com/movie/bronson,9,'MartinB',2009-10-13,'Anyone who doesn t like this movie simply ju...,0,1
2,https://www.metacritic.com/movie/bronson,10,'Jaakko',2012-07-19,'Not sure what to think at this film at first...,1,1
3,https://www.metacritic.com/movie/bronson,6,'CapoR',2009-10-13,'Nicely portrayed but it lacks the elements t...,0,1
4,https://www.metacritic.com/movie/bronson,8,'OrwellB.',2009-10-10,'Bronson is more than entertainment. It is ar...,0,0


- Dtypes:
url                  object
idvscore             object
reviewer             object
dateP        datetime64[ns]
Rev                  object
thumbsUp             object
thumbsTot            object
dtype: object
- Fraction of missing values (top 15):
thumbsUp     0.011199
thumbsTot    0.011187
Rev          0.010677
dateP        0.010677
idvscore     0.010649
reviewer     0.000000
url          0.000000
dtype: float64

Overview of ExpertReviews after cleaning
- Shape: (238973, 5)
- First rows:


,url,idvscore,reviewer,dateP,Rev
0,https://www.metacritic.com/movie/bronson,100.0,"""Andrew O'Hehir""",NaT,'Bronson owes a little or a lot to Kubrick s ...
1,https://www.metacritic.com/movie/bronson,90.0,'A.O. Scott',NaT,'Bronson invites you to admire its protagonis...
2,https://www.metacritic.com/movie/bronson,90.0,Unknown Reviewer,NaT,'Whether it s Peterson/Bronson s more theatri...
3,https://www.metacritic.com/movie/bronson,83.0,'Noel Murray',NaT,'There are two Bronsons on display here: the ...
4,https://www.metacritic.com/movie/bronson,80.0,'Joshua Rothkopf',NaT,'Refn has somehow found his way to an authent...


- Dtypes:
url                 object
idvscore           float64
reviewer            object
dateP       datetime64[ns]
Rev                 object
dtype: object
- Fraction of missing values (top 15):
dateP       0.437221
Rev         0.000008
idvscore    0.000008
url         0.000000
reviewer    0.000000
dtype: float64



In [23]:
# 3. Clean the meta_file_clean table --------------------------------

if "RelDate" in meta_file_clean.columns:
    meta_file_clean["RelDate"] = pd.to_datetime(
        meta_file_clean["RelDate"], errors="coerce"
    )

quick_overview(meta_file_clean, "meta_file_clean after cleaning")


Overview of meta_file_clean after cleaning
- Shape: (11364, 13)
- First rows:


,url,title,studio,rating,runtime,cast,director,genre,summary,awards,metascore,userscore,RelDate
0,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,Hotwire Productions,| Not Rated,83.0,NaN,Lynn Hershman-Leeson,Documentary,NaN,NaN,70,NaN,2011-06-01
1,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,Paramount Pictures,| PG-13,104.0,"John Gallagher Jr.,John Goodman,Mary Elizabeth...",Dan Trachtenberg,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror","Waking up from a car accident, a young woman (...","#18MostDiscussedMovieof2016 , #1MostSharedMovi...",76,7.7,2016-03-11
2,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,Click Star,| R,82.0,"Jonah Hill,Morgan Freeman,Paz Vega",Brad Silberling,"Drama,Comedy,Romance",While researching a role as a supermarket mana...,NaN,54,5.8,2006-12-01
3,https://www.metacritic.com/movie/10-years,10 Years,Anchor Bay Entertainment,| R,100.0,"Channing Tatum,Chris Pratt,Jenna Dewan",Jamie Linden,"Drama,Comedy,Romance",NaN,NaN,61,6.9,2012-09-14
4,https://www.metacritic.com/movie/100-bloody-acres,100 Bloody Acres,Music Box Films,| Not Rated,91.0,NaN,Cameron Cairnes,"Horror,Comedy",Reg and Lindsay run an organic fertilizer busi...,NaN,63,7.5,2013-06-28


- Dtypes:
url                  object
title                object
studio               object
rating               object
runtime             float64
cast                 object
director             object
genre                object
summary              object
awards               object
metascore             int64
userscore           float64
RelDate      datetime64[ns]
dtype: object
- Fraction of missing values (top 15):
awards       0.613956
summary      0.518919
cast         0.325766
userscore    0.185234
rating       0.093893
studio       0.030799
runtime      0.022439
genre        0.001760
director     0.001232
title        0.000000
url          0.000000
metascore    0.000000
RelDate      0.000000
dtype: float64



# New attempt of correctly clean the data!!!!!!!!!!


# Metacritic project – EDA & data cleaning

In this notebook we load the four Metacritic datasets, do some basic EDA,  
and then apply the full cleaning pipeline we built in the Database Management course.

The main steps:

1. Load raw Excel files from the `datasets` folder.
2. Quick EDA (shapes, columns, missing values).
3. Clean and standardise:
   - sales dates and box office numbers  
   - review dates and numeric columns  
   - reviewer names and IDs  
   - movie names / URLs and movie IDs
4. Return cleaned DataFrames that are ready for modelling (regression, KNN, etc.).


In [24]:
import os
import re
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # show all outputs in a cell


## 1. Load raw data

We assume this notebook lives in the project root (the `bjj` folder)  
and that all four Excel files are in the subfolder `datasets/`:

- `UserReviews.xlsx`
- `ExpertReviews.xlsx`
- `sales.xlsx`
- `metaClean43Brightspace.xlsx`


In [25]:
# Path to the datasets folder (relative to the notebook)
data_dir = os.path.join(os.getcwd(), "datasets")
data_dir
os.listdir(data_dir)


'c:\\Users\\jeffr\\OneDrive\\Documenten\\Digital Drive Business\\AI methods for business\\project map\\bjj\\datasets'

['ExpertReviews.xlsx',
 'metaClean43Brightspace.xlsx',
 'sales.xlsx',
 'UserReviews.xlsx']

In [28]:
# Load the four Excel files
UserReviews_raw = pd.read_excel(os.path.join(data_dir, "UserReviews.xlsx"))
ExpertReviews_raw = pd.read_excel(os.path.join(data_dir, "ExpertReviews.xlsx"))
sales_raw       = pd.read_excel(os.path.join(data_dir, "sales.xlsx"))
meta_raw        = pd.read_excel(os.path.join(data_dir, "metaClean43Brightspace.xlsx"))

UserReviews_raw.shape, ExpertReviews_raw.shape, sales_raw.shape, meta_raw.shape


((319662, 7), (238973, 5), (30612, 16), (11364, 13))

## 2. Quick EDA on raw data

We first look at:

- column names
- number of rows / columns
- a few first rows
- missing values

This is just to understand what we are cleaning.


In [29]:
def print_basic_info(name, df):
    print(f"\n{name}")
    print("-" * len(name))
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    print("\nHead():")
    display(df.head(3))
    print("\nMissing values per column:")
    display(df.isna().sum().sort_values(ascending=False).head(15))

print_basic_info("UserReviews_raw", UserReviews_raw)
print_basic_info("ExpertReviews_raw", ExpertReviews_raw)
print_basic_info("sales_raw", sales_raw)
print_basic_info("meta_raw", meta_raw)



UserReviews_raw
---------------
Shape: (319662, 7)
Columns: ['url', 'idvscore', 'reviewer', 'dateP', 'Rev', 'thumbsUp', 'thumbsTot']

Head():


,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot
0,https://www.metacritic.com/movie/bronson,8,'Longbottom94',"'Apr 25, 2013'",'Many have dismissed this film for not explor...,2,2
1,https://www.metacritic.com/movie/bronson,9,'MartinB',"'Oct 13, 2009'",'Anyone who doesn t like this movie simply ju...,0,1
2,https://www.metacritic.com/movie/bronson,10,'Jaakko',"'Jul 19, 2012'",'Not sure what to think at this film at first...,1,1



Missing values per column:


thumbsUp     3580
thumbsTot    3576
Rev          3413
dateP        3413
reviewer     3407
idvscore     3404
url             0
dtype: int64


ExpertReviews_raw
-----------------
Shape: (238973, 5)
Columns: ['url', 'idvscore', 'reviewer', 'dateP', 'Rev']

Head():


...

Unnamed: 8                  30612
production_budget           26132
opening_weekend             19683
avg run per theatre         19660
theatre_count               19649
domestic_box_office         18728
keywords                    18095
worldwide_box_office         9037
international_box_office     9037
runtime                      6053
creative_type                3945
genre                        1704
title                           8
year                            0
release_date                    0
dtype: int64


meta_raw
--------
Shape: (11364, 13)
Columns: ['url', 'title', 'studio', 'rating', 'runtime', 'cast', 'director', 'genre', 'summary', 'awards', 'metascore', 'userscore', 'RelDate']

Head():


,url,title,studio,rating,runtime,cast,director,genre,summary,awards,metascore,userscore,RelDate
0,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,Hotwire Productions,| Not Rated,83.0,NaN,Lynn Hershman-Leeson,Documentary,NaN,NaN,70,NaN,2011-06-01
1,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,Paramount Pictures,| PG-13,104.0,"John Gallagher Jr.,John Goodman,Mary Elizabeth...",Dan Trachtenberg,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror","Waking up from a car accident, a young woman (...","#18MostDiscussedMovieof2016 , #1MostSharedMovi...",76,7.7,2016-03-11
2,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,Click Star,| R,82.0,"Jonah Hill,Morgan Freeman,Paz Vega",Brad Silberling,"Drama,Comedy,Romance",While researching a role as a supermarket mana...,NaN,54,5.8,2006-12-01



Missing values per column:


awards       6977
summary      5897
cast         3702
userscore    2105
rating       1067
studio        350
runtime       255
genre          20
director       14
title           0
url             0
metascore       0
RelDate         0
dtype: int64

## 3. Helper functions for safe column access and numeric cleaning

These come from the DBM cleaning script and make the code more robust:

- `safe_column_access` – check if a column exists
- `get_numeric_value` – turn messy text into numbers
- `clean_numeric_columns` – apply numeric cleaning to multiple columns
- `calculate_total_box_office` – sum domestic + international box office
- `calculate_thumbs_down` – create `thumbsDown = thumbsTot - thumbsUp`


In [30]:
def safe_column_access(df, column_name, default=np.nan):
    """Return a column if it exists, otherwise a Series filled with a default value."""
    if column_name in df.columns:
        return df[column_name]
    else:
        return pd.Series([default] * len(df), index=df.index)

def get_numeric_value(text_series):
    """
    Clean a text column and convert it to numeric.
    Removes commas, currency symbols and anything non-numeric.
    """
    clean = (
        text_series.astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("$", "", regex=False)
        .str.replace("€", "", regex=False)
        .str.replace(r"[^\d\.\-]", "", regex=True)
    )
    return pd.to_numeric(clean, errors="coerce")

def clean_numeric_columns(df, columns):
    """Apply numeric cleaning to a list of columns."""
    for col in columns:
        if col in df.columns:
            df[col] = get_numeric_value(df[col])

def calculate_total_box_office(sales_df):
    """
    Create a single numeric column for total box office.
    Uses existing numeric columns if they are present.
    """
    print("Calculating total box office sales...")
    for col in [
        "domestic_box_office",
        "international_box_office",
        "worldwide_box_office",
        "production_budget",
        "opening_weekend",
    ]:
        if col in sales_df.columns:
            sales_df[col] = get_numeric_value(sales_df[col])

    if "worldwide_box_office" in sales_df.columns:
        sales_df["total_box_office"] = sales_df["worldwide_box_office"]
    else:
        sales_df["total_box_office"] = (
            safe_column_access(sales_df, "domestic_box_office", 0).fillna(0)
            + safe_column_access(sales_df, "international_box_office", 0).fillna(0)
        )

def calculate_thumbs_down(df):
    """Create thumbsDown from thumbsTot and thumbsUp if both are available."""
    if all(col in df.columns for col in ["thumbsUp", "thumbsTot"]):
        df["thumbsUp"] = get_numeric_value(df["thumbsUp"])
        df["thumbsTot"] = get_numeric_value(df["thumbsTot"])
        df["thumbsDown"] = df["thumbsTot"] - df["thumbsUp"]


## 4. Date cleaning

We want:

- sales release dates as proper `datetime`
- review dates as `datetime`
- meta data release dates as `datetime`

The idea is the same as in the DBMS script:
first try a direct parse, then fall back to a combination of year + date text.


In [31]:
def clean_date_text(series):
    """
    Try to parse messy date strings into real dates.
    Falls back to NaT when parsing fails.
    """
    # Strip whitespace
    text = series.astype(str).str.strip()

    # Replace obvious non-dates
    repl = ["nan", "none", "null", "", "not yet released", "tbd", "unknown"]
    text = text.str.lower().replace(repl, np.nan)

    # Let pandas try to parse (it will try many formats)
    clean_dates = pd.to_datetime(text, errors="coerce")

    return clean_dates

def clean_sales_dates(sales_df):
    """
    Create a clean release date column for the sales data.
    Uses 'release_date' and, as backup, 'year'.
    """
    print("Cleaning sales release dates...")

    release_date = safe_column_access(sales_df, "release_date")
    year_col = safe_column_access(sales_df, "year")

    clean_dates = clean_date_text(release_date)

    # If parsing failed but we at least have a year, try year-only
    missing_mask = clean_dates.isna() & year_col.notna()
    if missing_mask.any():
        backup = pd.to_datetime(year_col[missing_mask].astype(str), format="%Y", errors="coerce")
        clean_dates.loc[missing_mask] = backup

    sales_df["releasedate"] = clean_dates
    return sales_df["releasedate"]

def clean_expert_review_dates(df):
    """Standardize expert review dates."""
    if "dateP" in df.columns:
        df["expert_review_date"] = pd.to_datetime(df["dateP"], errors="coerce")
    elif "date" in df.columns:
        df["expert_review_date"] = pd.to_datetime(df["date"], errors="coerce")
    else:
        df["expert_review_date"] = pd.NaT

def clean_user_review_dates(df):
    """Standardize user review dates."""
    if "dateP" in df.columns:
        # original script formatted as YYYY-MM-DD string, then back to datetime
        df["dateP"] = pd.to_datetime(df["dateP"], errors="coerce").dt.strftime("%Y-%m-%d")
        df["user_review_date"] = pd.to_datetime(df["dateP"], errors="coerce")
    else:
        df["user_review_date"] = pd.NaT

def standardize_meta_dates(meta_df):
    """
    Standardise release date in the meta dataset.
    In your original file this was mostly cleaned already, but we enforce datetime.
    """
    # Try a few common column names – adjust if your actual column name is different
    for col in ["RelDate", "release_date", "releasedate"]:
        if col in meta_df.columns:
            meta_df["releasedate"] = clean_date_text(meta_df[col])
            return meta_df["releasedate"]

    # Fallback: no clear date column, create empty
    meta_df["releasedate"] = pd.NaT
    return meta_df["releasedate"]

def create_movie_date_mapping(sales_df, meta_df):
    """
    Build a mapping from cleaned movie name to release date using
    both sales and meta datasets.
    """
    # We will fill this mapping later once we have cleaned movie names,
    # so for now this returns an empty dict. We keep the function for structure.
    return {}


## 5. Reviewer names and IDs

We reuse the logic from DBMS:

- make sure reviewer names are never empty
- optionally lower-case or strip noise
- create `UserID` and `ExpertID`
- create sequential IDs for each review


In [32]:
def clean_reviewer_name(name):
    """Ensure reviewer names are never null or empty."""
    if pd.isna(name) or name is None:
        return "Unknown Reviewer"
    str_name = str(name).strip()
    if str_name in ["", "nan", "NaN", "None", "null"]:
        return "Unknown Reviewer"
    return str_name

def clean_reviewer_column(df):
    """Apply reviewer name cleaning to a DataFrame that has a 'reviewer' column."""
    if "reviewer" in df.columns:
        df["reviewer"] = df["reviewer"].apply(clean_reviewer_name)

def create_user_ids(UserReviews):
    """Create UserID for user reviews (group by reviewer name)."""
    if "reviewer" in UserReviews.columns:
        clean_reviewer_column(UserReviews)
        UserReviews["UserID"] = UserReviews["reviewer"].astype(str).str.strip().replace("", "Unknown Reviewer")
        UserReviews["UserID"] = UserReviews.groupby("UserID").ngroup() + 1
    else:
        UserReviews["reviewer"] = "Unknown Reviewer"
        UserReviews["UserID"] = range(1, len(UserReviews) + 1)

def create_expert_ids(ExpertReviews):
    """Create ExpertID for expert reviews (group by reviewer name)."""
    if "reviewer" in ExpertReviews.columns:
        clean_reviewer_column(ExpertReviews)
        ExpertReviews["ExpertID"] = ExpertReviews["reviewer"].astype(str).str.strip().replace("", "Unknown Reviewer")
        ExpertReviews["ExpertID"] = ExpertReviews.groupby("ExpertID").ngroup() + 1
    else:
        ExpertReviews["reviewer"] = "Unknown Reviewer"
        ExpertReviews["ExpertID"] = range(1, len(ExpertReviews) + 1)

def create_review_ids(df, id_column):
    """Create simple sequential IDs for each row."""
    df[id_column] = range(1, len(df) + 1)

def create_all_ids(UserReviews, ExpertReviews):
    """Run all ID creation steps."""
    print("Creating unique IDs.")
    create_user_ids(UserReviews)
    create_expert_ids(ExpertReviews)
    create_review_ids(UserReviews, "UserReviewID")
    create_review_ids(ExpertReviews, "ExpertReviewID")


## 6. Movie name cleaning and movie IDs

Now we fix the movie names to avoid duplicates caused by spelling / url differences.

Steps (same idea as the DBMS script):

1. Use the `url` column as base, or fall back to a title column.
2. Clean the text:
   - lowercase
   - remove trailing slashes and file extensions
   - remove year, brackets, common spam words like "watch", "online", etc.
3. Collect all movie references from all datasets.
4. Create a movie dimension table with a unique `movie_id` per cleaned name.
5. Add `movie_id` back to each dataset.


In [33]:
def clean_movie_name(text):
    """Make movie names consistent for matching across datasets."""
    if pd.isna(text) or text is None:
        return ""
    name = str(text).lower().strip().rstrip("/")
    name = name.rsplit("/", 1)[-1]          # take last part after slash (for URLs)
    name = name.replace("-", " ").replace("_", " ")
    # remove brackets content
    name = re.sub(r"\([^)]*\)|\[[^\]]*\]", "", name)
    # remove year at end like "(2019)" or "2019"
    name = re.sub(r"[\(\[]\s*(?:19|20)\d{2}[^)\]]*\s*[\)\]]\s*$|(?:19|20)\d{2}\s*$", "", name)
    # keep only letters, digits, spaces, hyphen and apostrophe
    name = re.sub(r"[^a-z0-9\s\-']", "", name)
    # collapse spaces
    name = re.sub(r"\s+", " ", name).strip()
    return name

def extract_title_from_url(text):
    """
    Extract a nice display title from a URL or raw text,
    mainly for presentation (not for matching).
    """
    if pd.isna(text) or text is None:
        return "Unknown Movie"
    original_text = str(text).strip()
    # If this does not look like a URL, clean it and capitalise
    if not any(ind in original_text.lower() for ind in ["http", "www.", ".com", ".net", ".org"]):
        title = original_text
        title = re.sub(r"\([^)]*\)|\[[^\]]*\]", "", title)
        title = re.sub(r"[\(\[]\s*(?:19|20)\d{2}[^)\]]*\s*[\)\]]\s*$|(?:19|20)\d{2}\s*$", "", title)
        title = re.sub(r"[^a-zA-Z0-9\s]", "", title)
        title = re.sub(r"\s+", " ", title).strip()
        if title:
            title = " ".join(word.capitalize() for word in title.split())
            return title
        else:
            return "Unknown Movie"

    # URL case
    url_text = original_text.lower()
    if "/" in url_text:
        path_part = url_text.split("/")[-1]
    else:
        path_part = url_text
    path_part = re.sub(r"\.html?$", "", path_part)
    path_part = re.sub(r"\.php$", "", path_part)
    title = path_part.replace("-", " ").replace("_", " ")
    # remove square brackets etc
    title = re.sub(r"\[[^\]]*\]", "", title)
    title = re.sub(r"[^a-zA-Z0-9\s]", "", title)
    title = re.sub(r"\s+", " ", title).strip()
    if title:
        title = " ".join(word.capitalize() for word in title.split())
        return title
    else:
        return "Unknown Movie"

def collect_all_movie_records(datasets, dataset_configs):
    """Collect all movie names from all datasets for ID generation."""
    all_movie_records = []
    for df_name, df in datasets.items():
        config = dataset_configs.get(df_name, {})
        source_col = config.get("source_column", "url")
        if source_col not in df.columns:
            continue

        cleaned = df[source_col].apply(clean_movie_name)
        display_titles = df[source_col].apply(extract_title_from_url)

        temp = pd.DataFrame(
            {
                "dataset": df_name,
                "original_text": df[source_col],
                "cleaned_name": cleaned,
                "display_title": display_titles,
            }
        )
        all_movie_records.append(temp)

    if not all_movie_records:
        return pd.DataFrame(columns=["dataset", "original_text", "cleaned_name", "display_title"])

    return pd.concat(all_movie_records, ignore_index=True)

def create_movie_dimension_table(all_movie_records):
    """Create the movie dimension table with unique movie IDs."""
    movie_key = (
        all_movie_records[["cleaned_name", "display_title"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    movie_key = movie_key[movie_key["cleaned_name"] != ""].reset_index(drop=True)
    movie_key["movie_id"] = range(1, len(movie_key) + 1)
    # Reorder columns
    movie_key = movie_key[["movie_id", "cleaned_name", "display_title"]]
    return movie_key

def add_movie_ids_to_datasets(datasets, movie_key, dataset_configs):
    """Merge movie_id back into each dataset based on cleaned movie name."""
    movie_lookup = movie_key[["movie_id", "cleaned_name"]].copy()

    for df_name, df in datasets.items():
        config = dataset_configs.get(df_name, {})
        source_col = config.get("source_column", "url")

        if source_col not in df.columns:
            print(f"  Skipping {df_name}: column '{source_col}' not found.")
            continue

        tmp = df[source_col].apply(clean_movie_name)
        df["cleaned_movie_name"] = tmp

        df_merged = df.merge(
            movie_lookup,
            how="left",
            left_on="cleaned_movie_name",
            right_on="cleaned_name",
        )
        df["movie_id"] = df_merged["movie_id"]
        datasets[df_name] = df  # update

def create_movie_ids_fixed(datasets):
    """
    Wrapper to generate a consistent movie_id across all datasets.
    Returns the movie dimension table.
    """
    print("Generating consistent movie IDs across all datasets.")

    dataset_configs = {
        "UserReviews": {"source_column": "url"},
        "ExpertReviews": {"source_column": "url"},
        "sales_df": {"source_column": "url"},
        "meta_file_clean": {"source_column": "url"},
    }

    all_movie_records = collect_all_movie_records(datasets, dataset_configs)
    print(f"  Found {len(all_movie_records)} total movie references")

    movie_key = create_movie_dimension_table(all_movie_records)
    print(f"  Identified {len(movie_key)} unique movies")

    add_movie_ids_to_datasets(datasets, movie_key, dataset_configs)

    # Simple coverage info
    for name, df in datasets.items():
        total = len(df)
        mapped = df["movie_id"].notna().sum()
        print(f"  {name}: mapped {mapped}/{total} records to movie_ids")

    return movie_key


## 7. Put everything together: full cleaning pipeline

Now we combine the pieces in one function:

1. Copy raw DataFrames (so we keep the originals).
2. Clean sales dates and box office.
3. Clean review numeric columns and thumbs.
4. Clean review dates.
5. Create IDs for users, experts and reviews.
6. Generate consistent `movie_id`s and a movie dimension table.
7. Return cleaned DataFrames and the `movie_key` table.


In [35]:
def process_review_data(UserReviews, ExpertReviews):
    """Process numeric columns, thumbs and dates for review datasets."""
    print("Processing review data...")

    # Numeric columns in both review sets (idvscore etc.)
    numeric_cols_common = ["idvscore"]
    # User specific numeric columns
    user_numeric_extra = ["thumbsUp", "thumbsTot"]
    # LIWC etc: treat as numeric if present
    liwc_like_cols = [
        "WC", "Analytic", "Clout", "Authentic", "Tone",
        "posemo", "negemo"
    ]

    # UserReviews
    user_numeric = [c for c in numeric_cols_common + user_numeric_extra + liwc_like_cols if c in UserReviews.columns]
    clean_numeric_columns(UserReviews, user_numeric)
    calculate_thumbs_down(UserReviews)
    clean_user_review_dates(UserReviews)

    # ExpertReviews
    expert_numeric = [c for c in numeric_cols_common + liwc_like_cols if c in ExpertReviews.columns]
    clean_numeric_columns(ExpertReviews, expert_numeric)
    clean_expert_review_dates(ExpertReviews)

def process_all_datasets_for_ai(UserReviews_raw, ExpertReviews_raw, sales_raw, meta_raw):
    """
    High level function that applies the full DBM cleaning logic
    but returns DataFrames for AI modelling instead of loading into a database.
    """
    # Work on copies so we never change the raw input
    UserReviews = UserReviews_raw.copy()
    ExpertReviews = ExpertReviews_raw.copy()
    sales_df     = sales_raw.copy()
    meta_file_clean = meta_raw.copy()

    print("Cleaning sales data...")
    clean_sales_dates(sales_df)
    calculate_total_box_office(sales_df)

    process_review_data(UserReviews, ExpertReviews)
    create_all_ids(UserReviews, ExpertReviews)

    print("Standardizing meta release dates...")
    standardize_meta_dates(meta_file_clean)

    print("Generating movie IDs...")
    datasets = {
        "UserReviews": UserReviews,
        "ExpertReviews": ExpertReviews,
        "sales_df": sales_df,
        "meta_file_clean": meta_file_clean,
    }
    movie_key = create_movie_ids_fixed(datasets)

    print("\nBasic data processing complete!")
    return datasets, movie_key


## 8. Run the cleaning and inspect results

Now we actually run the pipeline and check:

- cleaned shapes
- head of the movie dimension table
- example of a cleaned dataset (e.g. `UserReviews`).


In [36]:
datasets_clean, movie_key = process_all_datasets_for_ai(
    UserReviews_raw,
    ExpertReviews_raw,
    sales_raw,
    meta_raw,
)

UserReviews_clean = datasets_clean["UserReviews"]
ExpertReviews_clean = datasets_clean["ExpertReviews"]
sales_clean = datasets_clean["sales_df"]
meta_clean = datasets_clean["meta_file_clean"]

UserReviews_clean.shape, ExpertReviews_clean.shape, sales_clean.shape, meta_clean.shape, movie_key.shape


Cleaning sales data...
Cleaning sales release dates...
Calculating total box office sales...
Processing review data...
Creating unique IDs.
Standardizing meta release dates...
Generating movie IDs...
Generating consistent movie IDs across all datasets.
  Found 600611 total movie references
  Identified 36660 unique movies
  UserReviews: mapped 319071/319662 records to movie_ids
  ExpertReviews: mapped 238820/238973 records to movie_ids
  sales_df: mapped 30597/30612 records to movie_ids
  meta_file_clean: mapped 11355/11364 records to movie_ids

Basic data processing complete!


((319662, 13), (238973, 10), (30612, 20), (11364, 16), (36660, 3))

In [37]:
print("Movie dimension table (movie_key):")
movie_key.head()


Movie dimension table (movie_key):


,movie_id,cleaned_name,display_title
0,1,bronson,Bronson
1,2,disgrace,Disgrace
2,3,a screaming man,A Screaming Man
3,4,lovers of hate,Lovers Of Hate
4,5,phil ochs there but for fortune,Phil Ochs There But For Fortune


In [38]:
print_basic_info("UserReviews_clean", UserReviews_clean)



UserReviews_clean
-----------------
Shape: (319662, 13)
Columns: ['url', 'idvscore', 'reviewer', 'dateP', 'Rev', 'thumbsUp', 'thumbsTot', 'thumbsDown', 'user_review_date', 'UserID', 'UserReviewID', 'cleaned_movie_name', 'movie_id']

Head():


,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot,thumbsDown,user_review_date,UserID,UserReviewID,cleaned_movie_name,movie_id
0,https://www.metacritic.com/movie/bronson,8.0,'Longbottom94',2013-04-25,'Many have dismissed this film for not explor...,2.0,2.0,0.0,2013-04-25,40545,1,bronson,1.0
1,https://www.metacritic.com/movie/bronson,9.0,'MartinB',2009-10-13,'Anyone who doesn t like this movie simply ju...,0.0,1.0,1.0,2009-10-13,43490,2,bronson,1.0
2,https://www.metacritic.com/movie/bronson,10.0,'Jaakko',2012-07-19,'Not sure what to think at this film at first...,1.0,1.0,0.0,2012-07-19,30354,3,bronson,1.0



Missing values per column:


thumbsDown            3582
thumbsUp              3581
thumbsTot             3581
dateP                 3413
Rev                   3413
user_review_date      3413
idvscore              3409
movie_id               591
url                      0
reviewer                 0
UserID                   0
UserReviewID             0
cleaned_movie_name       0
dtype: int64